In [9]:
import pandas as pd
import pymysql
connection = pymysql.connect(host = "localhost",user = "root",password = "password",database = "nutrition") #connecting with database
cursor = connection.cursor()

#### Obesity

#### 1.Top 5 regions with the highest average obesity levels in the most recent year(2022)

In [5]:
q1 = """
SELECT Region, AVG(CASE 
                      WHEN obesity_level = 'High' THEN 1
                      WHEN obesity_level = 'Moderate' THEN 0.5
                      ELSE 0  -- Assuming 'Low' or other levels are given a value of 0
                    END) AS avg_obesity_level
FROM obesity
WHERE Year = 2022
GROUP BY Region
ORDER BY avg_obesity_level DESC
LIMIT 5;
"""
cursor.execute(q1)
results=cursor.fetchall()

In [11]:
results_df = pd.read_sql_query(q1, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\4193058502.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q1, connection)


,Region,avg_obesity_level
0,Western Pacific,0.31250
1,Americas,0.20270
2,Eastern Mediterranean,0.16288
3,Europe,0.09856
4,Special case,0.05303


#### 2.Top 5 countries with highest obesity estimates

In [14]:
q2 = """
SELECT Country, Mean_Estimate
FROM obesity
ORDER BY Mean_Estimate DESC
LIMIT 5;
"""
cursor.execute(q2)
results=cursor.fetchall()

In [16]:
results_df = pd.read_sql_query(q2, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\4292420737.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q2, connection)


,Country,Mean_Estimate
0,American Samoa,80.6093
1,American Samoa,80.3169
2,American Samoa,79.9699
3,American Samoa,79.6005
4,American Samoa,79.2050


#### 3.Obesity trend in India over the years(Mean_estimate)

In [19]:
q3 = """
SELECT Year,Country, Mean_Estimate
FROM obesity
WHERE Country = 'India'
GROUP BY Year, Mean_Estimate
ORDER BY Year;
"""
cursor.execute(q3)
results=cursor.fetchall()

In [21]:
results_df = pd.read_sql_query(q3, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\1242920312.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q3, connection)


,Year,Country,Mean_Estimate
0,2012,India,1.09485
1,2012,India,1.38786
2,2012,India,1.46895
3,2012,India,1.65354
4,2012,India,1.83549
...,...,...,...
127,2022,India,4.95572
128,2022,India,5.19603
129,2022,India,5.43879
130,2022,India,7.20651


In [23]:
q3 = """
SELECT Year,Country, AVG(Mean_Estimate) AS avg_obesity_estimate
FROM obesity
WHERE Country = 'India'
GROUP BY Year
ORDER BY Year;
"""
cursor.execute(q3)
results=cursor.fetchall()

In [25]:
results_df = pd.read_sql_query(q3, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\1242920312.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q3, connection)


,Year,Country,avg_obesity_estimate
0,2012,India,2.473178
1,2013,India,2.634653
2,2014,India,2.798748
3,2015,India,2.972814
4,2016,India,3.159577
5,2017,India,3.362175
6,2018,India,3.579292
7,2019,India,3.809013
8,2020,India,4.051149
9,2021,India,4.307815


#### 4.Average obesity by gender

In [28]:
q4 = """
SELECT Gender, AVG(Mean_Estimate) AS avg_obesity_estimate
FROM obesity
GROUP BY Gender
ORDER BY avg_obesity_estimate DESC;
"""
cursor.execute(q4)
results=cursor.fetchall()

In [30]:
results_df = pd.read_sql_query(q4, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\2691301739.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q4, connection)


,Gender,avg_obesity_estimate
0,Both,12.781970
1,Male,12.776696
2,Female,12.724456


#### 5.Country count by obesity level category and age group

In [33]:
q5 = """
SELECT obesity_level, age_group, COUNT(DISTINCT Country) AS country_count
FROM obesity
GROUP BY obesity_level, age_group
ORDER BY obesity_level, age_group;
"""
cursor.execute(q5)
results=cursor.fetchall()
#DISTINCT as we have multiple values for the same country

In [35]:
results_df = pd.read_sql_query(q5, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\2830943129.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q5, connection)


,obesity_level,age_group,country_count
0,High,Adult,94
1,High,Children,11
2,Low,Adult,176
3,Low,Children,210
4,Moderate,Adult,99
5,Moderate,Children,26


#### 6.Top 5 countries least reliable countries(with highest CI_Width) and Top 5 most consistent countries (smallest average CI_Width)

In [38]:
q6 = """
(
    -- Least Reliable Countries (Top 5 with highest CI_Width)
    SELECT Country, MAX(CI_Width) AS ci_width, 'Least Reliable' AS category
    FROM obesity
    GROUP BY Country
    ORDER BY ci_width DESC
    LIMIT 5
)

UNION ALL

(
    -- Most Consistent Countries (Top 5 with smallest average CI_Width)
    SELECT Country, AVG(CI_Width) AS ci_width, 'Most Consistent' AS category
    FROM obesity
    GROUP BY Country
    ORDER BY ci_width ASC
    LIMIT 5
);
"""
cursor.execute(q6)
results = cursor.fetchall()

In [40]:
results_df = pd.read_sql_query(q6, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\1011017246.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q6, connection)


,Country,ci_width,category
0,Haiti,40.554062,Least Reliable
1,Bermuda,40.079681,Least Reliable
2,Puerto Rico,40.032200,Least Reliable
3,American Samoa,39.954140,Least Reliable
4,Grenada,39.696850,Least Reliable
5,Global,0.708952,Most Consistent
6,Low & Middle Income,0.861059,Most Consistent
7,South-East Asia Region,1.017775,Most Consistent
8,India,1.140133,Most Consistent
9,Japan,1.155902,Most Consistent


#### 7.Average obesity by age group

In [43]:
q7 = """
SELECT age_group, AVG(Mean_Estimate) AS avg_obesity_estimate
FROM obesity
GROUP BY age_group
ORDER BY avg_obesity_estimate DESC;
"""
cursor.execute(q7)
results=cursor.fetchall()

In [45]:
results_df = pd.read_sql_query(q7, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\1307740939.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q7, connection)


,age_group,avg_obesity_estimate
0,Adult,21.833224
1,Children,9.736980


#### 8.Top 10 Countries with consistent low obesity (low average + low CI)over the years

In [48]:
q8 = """
SELECT Country, 
       AVG(Mean_Estimate) AS avg_obesity, 
       AVG(CI_Width) AS avg_ci_width,
       COUNT(DISTINCT Year) AS years_count
FROM obesity
GROUP BY Country
ORDER BY avg_obesity ASC, avg_ci_width ASC
LIMIT 10;
"""
cursor.execute(q8)
results=cursor.fetchall()

In [50]:
results_df = pd.read_sql_query(q8, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\4024566541.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q8, connection)


,Country,avg_obesity,avg_ci_width,years_count
0,Timor-Leste,0.994067,1.273569,11
1,Ethiopia,1.491995,1.589414,11
2,Burkina Faso,1.721496,1.531514,11
3,Rwanda,1.767194,2.360578,11
4,Bangladesh,1.976673,1.663507,11
5,Madagascar,1.984110,4.356886,11
6,Burundi,2.304665,5.198043,11
7,Malawi,2.391324,2.632713,11
8,Nepal,2.494255,3.098051,11
9,Eritrea,2.770560,5.870395,11


#### 9.Countries where female obesity exceeds male by large margin (same year)

In [53]:
q9 = """
SELECT 
    female.Country,
    female.Year,
    female.Mean_Estimate AS female_obesity_rate,
    male.Mean_Estimate AS male_obesity_rate
FROM 
    obesity AS female,
    obesity AS male
WHERE 
    female.Country = male.Country 
    AND female.Year = male.Year 
    AND female.Gender = 'Female' 
    AND male.Gender = 'Male' 
    AND (female.Mean_Estimate - male.Mean_Estimate) >= 5
ORDER BY 
    (female.Mean_Estimate - male.Mean_Estimate) DESC;
"""
cursor.execute(q9)
results=cursor.fetchall()

In [55]:
results_df = pd.read_sql_query(q9, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\3809503348.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q9, connection)


,Country,Year,female_obesity_rate,male_obesity_rate
0,American Samoa,2012,76.98380,23.72960
1,American Samoa,2012,76.98380,23.86650
2,American Samoa,2012,76.98380,24.11690
3,American Samoa,2013,77.29770,24.59920
4,American Samoa,2013,77.29770,24.72740
...,...,...,...,...
7373,Indonesia,2021,15.54430,10.53740
7374,Brazil,2017,27.25490,22.24940
7375,Denmark,2018,11.92180,6.91778
7376,Bosnia and Herzegovina,2021,25.89620,20.89270


#### 10.Global average obesity percentage per year

In [58]:
q10 = """
SELECT 
    Year,
    AVG(Mean_Estimate) AS global_avg_obesity_rate
FROM obesity
GROUP BY Year
ORDER BY Year;
"""
cursor.execute(q10)
results=cursor.fetchall()

In [60]:
results_df = pd.read_sql_query(q10, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\3182209104.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q10, connection)


,Year,global_avg_obesity_rate
0,2012,11.061970
1,2013,11.359638
2,2014,11.670344
3,2015,11.994231
4,2016,12.333907
5,2017,12.689997
6,2018,13.060297
7,2019,13.444604
8,2020,13.843002
9,2021,14.250923


### Malnutrition

### 11.Avg. malnutrition by age group

In [64]:
q11 = """
SELECT age_group, AVG(Mean_Estimate) AS avg_malnutrition_estimate
FROM malnutrition
GROUP BY age_group
ORDER BY avg_malnutrition_estimate DESC;
"""
cursor.execute(q11)
results=cursor.fetchall()

In [66]:
results_df = pd.read_sql_query(q11, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\3552536008.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q11, connection)


,age_group,avg_malnutrition_estimate
0,Adult,5.940619
1,Children,5.159062


#### 12.Top 5 countries with highest malnutrition(mean_estimate)

In [69]:
q12 = """
SELECT 
    Country,
    Year,
    Mean_Estimate
FROM malnutrition
ORDER BY Mean_Estimate DESC
LIMIT 5;
"""
cursor.execute(q12)
results=cursor.fetchall()

In [71]:
results_df = pd.read_sql_query(q12, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\2556301108.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q12, connection)


,Country,Year,Mean_Estimate
0,Eritrea,2012,35.8941
1,Eritrea,2013,35.6944
2,Eritrea,2014,35.4982
3,Eritrea,2015,35.3130
4,Eritrea,2016,35.1477


#### 13.Malnutrition trend in African region over the years

In [74]:
q13 = """
SELECT 
    Year,
    AVG(Mean_Estimate) AS average_malnutrition_rate
FROM malnutrition
WHERE Region = 'Africa'  
GROUP BY Year
ORDER BY Year;
"""
cursor.execute(q13)
results=cursor.fetchall()

In [76]:
results_df = pd.read_sql_query(q13, connection)
results_df


C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\4250644692.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q13, connection)


,Year,average_malnutrition_rate
0,2012,8.849655
1,2013,8.749857
2,2014,8.662762
3,2015,8.587719
4,2016,8.523125
5,2017,8.466465
6,2018,8.414722
7,2019,8.369700
8,2020,8.331320
9,2021,8.298626


#### 14.Gender-based average malnutrition

In [79]:
q14 = """
SELECT 
    Gender,
    AVG(Mean_Estimate) AS average_malnutrition_rate
FROM malnutrition
GROUP BY Gender;
"""
cursor.execute(q14)
results=cursor.fetchall()

In [81]:
results_df = pd.read_sql_query(q14, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\1041298341.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q14, connection)


,Gender,average_malnutrition_rate
0,Female,4.675483
1,Both,5.361553
2,Male,6.026317


#### 15.Malnutrition level-wise (average CI_Width by age group

In [84]:
q15 = """
SELECT 
    malnutrition_level,
    age_group,
    AVG(CI_Width) AS average_CI_Width
FROM malnutrition
GROUP BY 
    malnutrition_level,
    age_group;
"""
cursor.execute(q15)
results=cursor.fetchall()

In [86]:
results_df = pd.read_sql_query(q15, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\844104432.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q15, connection)


,malnutrition_level,age_group,average_CI_Width
0,Low,Adult,3.608206
1,High,Adult,13.289683
2,Moderate,Adult,11.206546
3,Low,Children,5.101265
4,Moderate,Children,6.089769
5,High,Children,3.198873


#### 16.Yearly malnutrition change in specific countries(India, Nigeria, Brazil)

In [89]:
q16 = """
SELECT 
    Country,
    Year,
    Mean_Estimate,
    LAG(Mean_Estimate) OVER (PARTITION BY Country ORDER BY Year) AS previous_year_malnutrition_rate,
    Mean_Estimate - LAG(Mean_Estimate) OVER (PARTITION BY Country ORDER BY Year) AS malnutrition_change
FROM malnutrition
WHERE  Country IN ('India', 'Nigeria', 'Brazil')  -- Filter for specific countries
ORDER BY Country, Year;
"""
cursor.execute(q16)
results=cursor.fetchall()

In [91]:
results_df = pd.read_sql_query(q16, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\2984656181.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q16, connection)


,Country,Year,Mean_Estimate,previous_year_malnutrition_rate,malnutrition_change
0,Brazil,2012,3.02790,NaN,NaN
1,Brazil,2012,2.58262,3.02790,-0.445280
2,Brazil,2012,2.54583,2.58262,-0.036790
3,Brazil,2012,2.69123,2.54583,0.145400
4,Brazil,2012,2.87519,2.69123,0.183960
...,...,...,...,...,...
391,Nigeria,2022,11.38430,12.82770,-1.443410
392,Nigeria,2022,18.53820,11.38430,7.153910
393,Nigeria,2022,9.61638,18.53820,-8.921800
394,Nigeria,2022,9.59384,9.61638,-0.022540


#### 17.Regions with lowest malnutrition averages

In [94]:
q17 = """
SELECT 
    Region,
    AVG(Mean_Estimate) AS average_malnutrition_rate
FROM malnutrition
GROUP BY Region
ORDER BY average_malnutrition_rate ASC;
"""
cursor.execute(q17)
results=cursor.fetchall()

In [96]:
results_df = pd.read_sql_query(q17, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\2743693007.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q17, connection)


,Region,average_malnutrition_rate
0,Europe,2.426759
1,Americas,3.174752
2,Western Pacific,3.220453
3,Eastern Mediterranean,6.661349
4,Special case,8.354200
5,Africa,8.502186
6,South-East Asia,13.283206


#### 18. Countries with increasing malnutrition

In [99]:
q18 = """
SELECT 
    Country,
    MIN(Year) AS early_year,
    MAX(Year) AS recent_year,
    MIN(Mean_Estimate) AS early_malnutrition_rate,
    MAX(Mean_Estimate) AS recent_malnutrition_rate,
    MAX(Mean_Estimate) - MIN(Mean_Estimate) AS malnutrition_change
FROM malnutrition
GROUP BY Country
HAVING MAX(Mean_Estimate) > MIN(Mean_Estimate)  -- Filtering where malnutrition has increased
ORDER BY malnutrition_change DESC;
"""
cursor.execute(q18)
results=cursor.fetchall()

In [101]:
results_df = pd.read_sql_query(q18, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\2101103698.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q18, connection)


,Country,early_year,recent_year,early_malnutrition_rate,recent_malnutrition_rate,malnutrition_change
0,Eritrea,2012,2022,7.86806,35.89410,28.026052
1,Ethiopia,2012,2022,7.55397,29.30410,21.750081
2,Somalia,2012,2022,4.68890,25.52170,20.832830
3,India,2012,2022,13.07350,31.90080,18.827341
4,Pakistan,2012,2022,8.21369,25.95060,17.736900
...,...,...,...,...,...,...
205,Cook Islands,2012,2022,0.44990,1.63716,1.187260
206,Peru,2012,2022,0.82517,1.98527,1.160100
207,American Samoa,2012,2022,0.24652,1.14696,0.900440
208,Niue,2012,2022,0.35336,1.24984,0.896480


#### 19.Min/Max malnutrition levels year-wise comparison

In [104]:
q19 = """
SELECT 
    Year,
    MIN(Mean_Estimate) AS min_malnutrition_rate,
    MAX(Mean_Estimate) AS max_malnutrition_rate
FROM malnutrition
GROUP BY Year;
"""
cursor.execute(q19)
results=cursor.fetchall()

In [106]:
results_df = pd.read_sql_query(q19, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\11920010.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q19, connection)


,Year,min_malnutrition_rate,max_malnutrition_rate
0,2021,0.18472,34.4292
1,2017,0.17929,34.9979
2,2016,0.17880,35.1477
3,2014,0.17777,35.4982
4,2020,0.18199,34.5784
5,2012,0.17531,35.8941
6,2019,0.18043,34.7160
7,2018,0.17980,34.8562
8,2022,0.18817,34.2620
9,2013,0.17642,35.6944


#### 20.High CI_Width flags for monitoring(CI_width > 5)

In [109]:
q20 = """
SELECT 
    Region,
    Country,
    Year,
    Mean_Estimate,
    CI_Width
FROM malnutrition
WHERE CI_Width > 5
ORDER BY CI_Width DESC;
"""
cursor.execute(q20)
results=cursor.fetchall()

In [111]:
results_df = pd.read_sql_query(q20, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\3059778940.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q20, connection)


,Region,Country,Year,Mean_Estimate,CI_Width
0,Africa,Senegal,2022,16.94180,26.70170
1,Africa,Senegal,2021,16.99200,25.67930
2,Africa,Senegal,2020,17.04640,24.58640
3,Africa,Mauritania,2022,16.08090,23.81050
4,Africa,Senegal,2019,17.10590,23.46690
...,...,...,...,...,...
9566,Europe,Iceland,2022,1.42837,5.00254
9567,Eastern Mediterranean,Jordan,2019,3.87659,5.00189
9568,Africa,Comoros,2018,3.27296,5.00156
9569,Western Pacific,Philippines,2021,7.55125,5.00152


#### 21.Obesity vs malnutrition comparison by country(any 5 countries)

In [114]:
q21 = """
SELECT 
    o.Country,
    AVG(o.Mean_Estimate) AS Average_Obesity,
    AVG(m.Mean_Estimate) AS Average_Malnutrition
FROM obesity o
JOIN malnutrition m ON o.Country = m.Country  -- Join the tables on the Country field
WHERE o.Country IN ('India', 'Nigeria', 'Brazil', 'Europe', 'Mexico')  -- Filtering for specific countries
GROUP BY o.Country;
"""
cursor.execute(q21)
results=cursor.fetchall()

In [116]:
results_df = pd.read_sql_query(q21, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\2066438355.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q21, connection)


,Country,Average_Obesity,Average_Malnutrition
0,Nigeria,4.467604,11.889187
1,India,3.429781,22.142361
2,Europe,11.853726,2.545576
3,Brazil,16.613028,2.685355
4,Mexico,20.062568,1.845167


#### 22.Gender-based disparity in both obesity and malnutrition

In [119]:
q22 = """
SELECT 
    o.Gender,
    AVG(o.Mean_Estimate) AS Average_Obesity,
    AVG(m.Mean_Estimate) AS Average_Malnutrition
FROM obesity o
JOIN malnutrition m ON o.Country = m.Country AND o.Gender = m.Gender  -- Join on both Country and Gender
GROUP BY o.Gender;
"""
cursor.execute(q22)
results=cursor.fetchall()

In [126]:
results_df = pd.read_sql_query(q22, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\1644330190.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q22, connection)


,Gender,Average_Obesity,Average_Malnutrition
0,Male,12.776696,6.026317
1,Female,12.724456,4.675483
2,Both,12.781970,5.361553


#### 23.Region-wise avg estimates side-by-side(Africa and America)

In [128]:
q23 = """
SELECT 
    o.Region,
    AVG(o.Mean_Estimate) AS Avg_Obesity,
    AVG(m.Mean_Estimate) AS Avg_Malnutrition
FROM obesity o
JOIN malnutrition m ON o.Country = m.Country   
WHERE o.Region IN ('Africa', 'Americas')  -- Filter for the specified regions
GROUP BY o.Region;
"""
cursor.execute(q23)
results=cursor.fetchall()

In [130]:
results_df = pd.read_sql_query(q23, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\2555742136.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q23, connection)


,Region,Avg_Obesity,Avg_Malnutrition
0,Africa,5.415204,8.502186
1,Americas,18.402103,3.174752


#### 24.Countries with obesity up & malnutrition down

In [133]:
q24 = """
SELECT 
    o.Country,
    MIN(o.Mean_Estimate) AS Min_Obesity,
    MAX(o.Mean_Estimate) AS Max_Obesity,
    MIN(m.Mean_Estimate) AS Min_Malnutrition,
    MAX(m.Mean_Estimate) AS Max_Malnutrition,
    COUNT(DISTINCT o.Year) AS Years_Tracked
FROM obesity o
JOIN  malnutrition m ON o.Country = m.Country 
GROUP BY o.Country;
"""
cursor.execute(q24)
results=cursor.fetchall()

In [134]:
results_df = pd.read_sql_query(q24, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\1757632553.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q24, connection)


,Country,Min_Obesity,Max_Obesity,Min_Malnutrition,Max_Malnutrition,Years_Tracked
0,Afghanistan,3.10666,21.7908,5.12294,15.94920,11
1,Africa,1.69413,15.3177,5.95334,14.92330,11
2,Albania,3.12137,29.8361,0.62724,3.31530,11
3,Algeria,10.16850,32.9296,3.33591,6.87127,11
4,American Samoa,10.73650,80.6093,0.24652,1.14696,11
...,...,...,...,...,...,...
205,Viet Nam,0.59484,14.2363,7.17039,20.83140,11
206,Western Pacific Region,2.83084,17.8769,3.62325,8.56831,11
207,Yemen,3.30097,16.8876,11.30990,20.76140,11
208,Zambia,1.39426,14.4270,3.04869,12.36540,11


In [137]:
q24 = """
SELECT 
    o.Country,
    MIN(o.Mean_Estimate) AS Min_Obesity,
    MAX(o.Mean_Estimate) AS Max_Obesity,
    MIN(m.Mean_Estimate) AS Min_Malnutrition,
    MAX(m.Mean_Estimate) AS Max_Malnutrition
FROM obesity o
JOIN  malnutrition m ON o.Country = m.Country
GROUP BY o.Country
HAVING 
    MAX(o.Mean_Estimate) > MIN(o.Mean_Estimate)  -- Obesity has increased
    AND MAX(m.Mean_Estimate) < MIN(m.Mean_Estimate);  -- Malnutrition has decreased
"""
cursor.execute(q24)
results=cursor.fetchall()

In [138]:
results_df = pd.read_sql_query(q24, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\1757632553.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q24, connection)


,Country,Min_Obesity,Max_Obesity,Min_Malnutrition,Max_Malnutrition


#### 25.Age-wise trend analysis

In [142]:
q25 = """
SELECT 
    o.age_group,
    o.Year,
    AVG(o.Mean_Estimate) AS Avg_Obesity,
    AVG(m.Mean_Estimate) AS Avg_Malnutrition
FROM obesity o
JOIN malnutrition m ON o.Country = m.Country 
GROUP BY o.age_group, o.Year;  
"""
cursor.execute(q25)
results=cursor.fetchall()

In [144]:
results_df = pd.read_sql_query(q25, connection)
results_df

C:\Users\PC1\AppData\Local\Temp\ipykernel_6428\1443583078.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results_df = pd.read_sql_query(q25, connection)


,age_group,Year,Avg_Obesity,Avg_Malnutrition
0,Adult,2020,23.262193,5.354451
1,Adult,2014,20.391706,5.354451
2,Adult,2019,22.757323,5.354451
3,Adult,2018,22.269403,5.354451
4,Adult,2021,23.777188,5.354451
5,Adult,2015,20.847086,5.354451
6,Adult,2017,21.790270,5.354451
7,Adult,2016,21.315222,5.354451
8,Adult,2012,19.517937,5.354451
9,Adult,2013,19.949265,5.354451
